In [16]:
## load joint info

import pandas as pd 

JOINT_CSV_FILENAME = '/var/data/MET2/activity.csv'

numJoints = 14
# prepare column names 
colNames = list()
colNames.append('ID')
for n in range(numJoints):
    colNames.append("x{}".format(n+1))
    colNames.append("y{}".format(n+1))    
for n in range(numJoints):
    colNames.append("v{}".format(n+1))
print(colNames)

joints=pd.read_csv(JOINT_CSV_FILENAME,header=None,names=colNames,index_col=0)
#joints.shape()
joints.head()

['ID', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4', 'x5', 'y5', 'x6', 'y6', 'x7', 'y7', 'x8', 'y8', 'x9', 'y9', 'x10', 'y10', 'x11', 'y11', 'x12', 'y12', 'x13', 'y13', 'x14', 'y14', 'v1', 'v2', 'v3', 'v4', 'v5', 'v6', 'v7', 'v8', 'v9', 'v10', 'v11', 'v12', 'v13', 'v14']


,x1,y1,x2,y2,x3,y3,x4,y4,x5,y5,...,v5,v6,v7,v8,v9,v10,v11,v12,v13,v14
ID,,,,,,,,,,,,,,,,,,,,,
image/office activities/walking about/1.png,113,172,113,147,137,117,119,117,123,153,...,0,0,0,0,0,0,0,0,0,0
image/office activities/walking about/10.png,187,196,184,159,177,115,205,110,203,152,...,0,0,0,0,0,0,0,0,0,0
image/office activities/walking about/11.jpg,73,294,73,227,66,160,79,153,96,212,...,0,0,0,0,0,1,1,1,0,0
image/office activities/walking about/12.jpg,196,188,200,145,198,104,170,100,176,147,...,0,0,0,0,0,0,0,0,0,0
image/office activities/walking about/13.jpg,102,189,86,136,131,110,131,118,131,162,...,0,0,0,1,1,0,0,0,0,0
